In [6]:
"""
dropout层会提高泛化能力，但是扔掉的东西太多了的话会导致收敛变慢
强行修改图片尺寸会对结果造成很不好的影响
"""

'\ndropout层会提高泛化能力，但是扔掉的东西太多了的话会导致收敛变慢\n强行修改图片尺寸会对结果造成很不好的影响\n'

In [7]:
from torchvision import transforms, datasets,models
from torch.utils.data import DataLoader
from torch.optim import Adam
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=0.1307, std=0.3081)

])
train_dataset = datasets.CIFAR10(root='../dataset', train=True, download=True, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)

test_dataset = datasets.CIFAR10(root='../dataset', train=False, download=True, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)




In [10]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet,self).__init__()
        self.relu = nn.ReLU()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=96, kernel_size=11, stride=4, padding=2),
            nn.BatchNorm2d(num_features=96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, padding=2),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.layer3 = nn.Sequential(nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, padding=1),
                                    nn.BatchNorm2d(num_features=384),
                                    nn.ReLU(),)
        self.layer4 = nn.Sequential(nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, padding=1),
                                    nn.BatchNorm2d(num_features=384),
                                    nn.ReLU(),)
        self.layer5 = nn.Sequential(nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, padding=1),
                                    nn.BatchNorm2d(num_features=256),
                                    nn.ReLU(),)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.fc1 = nn.Linear(in_features=256 * 6 * 6, out_features=256)
        self.dp1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(in_features=256, out_features=256)
        self.dp2 = nn.Dropout(0.1)
        self.fc3 = nn.Linear(in_features=256, out_features=10)


    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.pool(x)
        x = x.view(-1, 256 * 6 * 6)
        x = self.dp1(self.relu(self.fc1(x)))
        x = self.dp2(self.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

def init_weights(m):
    if isinstance(m, (nn.Conv2d, nn.Linear)):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.zero_()

model = models.alexnet(pretrained=True)
model = model.apply(init_weights)
model = model.to(device)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to E:\DeepLearing\torch_cache\hub\checkpoints\alexnet-owt-7be5be79.pth
  1%|▏         | 3.12M/233M [00:12<15:55, 252kB/s]


KeyboardInterrupt: 

In [4]:
optimizer = Adam(model.parameters(), lr=0.005)
criterion = nn.CrossEntropyLoss()


def train(num_epochs):
    model.train()
    losses = []
    epoches = torch.arange(1, num_epochs + 1)

    for epoch in range(num_epochs):
        running_loss = 0.0
        total = 0
        correct = 0
        for idx, (x, y) in enumerate(train_dataloader):
            optimizer.zero_grad()
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)

            loss.backward()
            optimizer.step()
            running_loss += loss.item() * x.size(0)
            total += y.size(0)
            correct += (y_pred.argmax(dim=1) == y).sum().item()
            if (idx + 1) % 100 == 0:
                print(
                    f"Epoch [{epoch + 1}/{num_epochs}],Batches[{idx + 1}/{len(train_dataloader)}], Loss: {running_loss / ((idx + 1) * (x.size(0))):.3f} , Accuracy: {(100 * correct / total):.2f}%")
        losses.append(running_loss / len(train_dataloader.dataset))
        print(f"第{epoch + 1}轮的训练精度为：{(100 * correct / total):.2f}%\n")
    plt.plot(epoches, losses)
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.show()


def test():
    model.eval()  # 设置模型为评估模式
    correct = 0
    total = 0

    with torch.no_grad():  # 不计算梯度
        for images, labels in test_dataloader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)  # 获取预测结果
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total:.2f}%')

In [5]:
if __name__ == '__main__':
    num_epochs = 10
    train(num_epochs)
    test()

RuntimeError: Given groups=1, weight of size [128, 1, 11, 11], expected input[128, 3, 224, 224] to have 1 channels, but got 3 channels instead